In [12]:
import sys, os; sys.path.append('..')
import random
import pyzx as zx
import logging
from fractions import Fraction
from pyzx.gadget_extract import *
Z = zx.VertexType.Z
X = zx.VertexType.X
SE = zx.EdgeType.SIMPLE
HE = zx.EdgeType.HADAMARD
# zx.settings.drawing_backend = 'matplotlib'

In [13]:
random.seed(1337)
g = zx.generate.cliffordT(8, 100, p_t=0.2)
zx.full_reduce(g)
g.normalize()

In [14]:
zx.draw(g)

In [15]:
g1 = g.copy()
gadgetize(g1)
zx.draw(g1)